# California Restaurant "Likes" Prediction Using Foursquare API and Machine Learning

**Oliver Ma** <br>
Capstone Project <br>
IBM Data Science Professional Certificate <br> 

## 1. Introduction

California boasts an incredibly diverse collection of restaurants catering to different palettes and appetites. A large part of marketing for a modern restaurant (or any company) is social media, where the number of "likes" that the company can receive will dictate its brand and image to the general public. <br>

For a new business owner (or existing company) to open a new restaurant in California, knowing ahead of time the potential social media image they can have would provide an excellent solution to the ever present business problem of uncertainty. In this case the uncertainty is regarding performance of social media presence. 
<br>

We can mitigate this uncertainty through leveraging data gathered from FourSquare's API, specifically, we are able to scrape "likes" data of different restaurants directly from the API as well as their location and category of cuisine. The question we will try to address is, how accurately can we predict the amount of "likes" a new restaurant opening in this region can expect to have based on the type of cuisine it will serve and which city in California it will open in. (For the purposes of this analysis, we will contain the geographical scope of analysis to three heavily populated cities in California, namely San Francisco, Los Angeles, and San Diego). 
<br>

Leveraging this data will solve the problem as it allows the new business owner (or existing company) to make preemptive business decisions regarding opening the restaurant in terms of whether it is feasible to open one in this region and expect good social media presence, what type of cuisine and which city of three would be the best. This project will analyze and model the data via machine learning through comparing both linear and logistic regressions to see which method will yield better predictive capabilities after training and testing. 
<br>

Let us begin by importing the necessary packages.



In [68]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim 

import requests
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


import folium 

from urllib.request import urlopen
from bs4 import BeautifulSoup


import matplotlib.pyplot as plt
import pylab as pl

from sklearn import linear_model
from sklearn.metrics import jaccard_similarity_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, r2_score
import itertools


print('Libraries imported.')

Libraries imported.


# 2. Data 

## 2.1 Data Scraping and Cleaning

In this section we will first retrieve the geographical coordinates of the three cities (San Francisco, Los Angeles, and San Diego). Then, we will leverage the FourSquare API to obtain URLs that lead to the raw data in JSON form. We will speerately scrape the raw data in these URLs in order to retrieve the following columns: "name", "categories", "latitude", "longitude". and "id" for each city. We can also provide another column ("city") to indicate which city the restaurants are from. 
<br>

It is important to note that the extracts are not of every restaurant in those cities but rather all of the restaurants within a 1000KM range of the geographical coordinates that geolocator was able to provide. However, the extraction from the FourSquare API actually obtains venue data so it will include venues other than restaurants such as concert halls, stores, libraries etc. As such, this means that the data will need to be further cleaned somewhat manually by removing all of the non-restaurant rows. Once this is complete, we have a shortened by cleaned list to pull "likes" data. The reason the cleaning takes precedence is mainly that pulling the "likes" data is the computing process which takes the longest time in this project so we want to make sure we are not pulling information that will end up being dropped anyways.

The "id" is an important column as it will allow us to further pull the "likes" from the API. We can retreive the "likes" based on the restaurant "id" and then append it to the data frame. Once this is complete, we finally name the dataframe 'raw_dataset' as it is the most complete compiled form before needing any processing for analysis via machine learning. 

In [2]:
address1 = 'San Francisco, California'

geolocator = Nominatim()
location1 = geolocator.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address1, latitude1, longitude1))


address2 = 'Los Angeles, California'

geolocator = Nominatim()
location2 = geolocator.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address2, latitude2, longitude2))

address3 = 'San Diego, California'

geolocator = Nominatim()
location3 = geolocator.geocode(address3)
latitude3 = location3.latitude
longitude3 = location3.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address3, latitude3, longitude3))


C:\Users\olive\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of San Francisco, California are 37.7792808, -122.4192363.


C:\Users\olive\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if sys.path[0] == '':


The geograpical coordinate of Los Angeles, California are 34.0536834, -118.2427669.


C:\Users\olive\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of San Diego, California are 32.7174209, -117.1627714.


In [3]:
CLIENT_ID = '1AJETP2F5JJMQ22BA33HNS2YNOF0QKT5HDAJNHBRMK04LQVC' # your Foursquare ID
CLIENT_SECRET = 'KEVIFVL2Y0BINL5HGRGRTWGOKUETUMVY5L124X1BJHVBDEOD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URLs
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude1, 
    longitude1, 
    radius, 
    LIMIT)


# create URLs
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude2, 
    longitude2, 
    radius, 
    LIMIT)


# create URLs
url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude3, 
    longitude3, 
    radius, 
    LIMIT)

print(url1, url2, url3)


Your credentails:
CLIENT_ID: 1AJETP2F5JJMQ22BA33HNS2YNOF0QKT5HDAJNHBRMK04LQVC
CLIENT_SECRET:KEVIFVL2Y0BINL5HGRGRTWGOKUETUMVY5L124X1BJHVBDEOD
https://api.foursquare.com/v2/venues/explore?&client_id=1AJETP2F5JJMQ22BA33HNS2YNOF0QKT5HDAJNHBRMK04LQVC&client_secret=KEVIFVL2Y0BINL5HGRGRTWGOKUETUMVY5L124X1BJHVBDEOD&v=20180605&ll=37.7792808,-122.4192363&radius=1000&limit=100 https://api.foursquare.com/v2/venues/explore?&client_id=1AJETP2F5JJMQ22BA33HNS2YNOF0QKT5HDAJNHBRMK04LQVC&client_secret=KEVIFVL2Y0BINL5HGRGRTWGOKUETUMVY5L124X1BJHVBDEOD&v=20180605&ll=34.0536834,-118.2427669&radius=1000&limit=100 https://api.foursquare.com/v2/venues/explore?&client_id=1AJETP2F5JJMQ22BA33HNS2YNOF0QKT5HDAJNHBRMK04LQVC&client_secret=KEVIFVL2Y0BINL5HGRGRTWGOKUETUMVY5L124X1BJHVBDEOD&v=20180605&ll=32.7174209,-117.1627714&radius=1000&limit=100


In [4]:
# scrape the data from the generated URLs

results1 = requests.get(url1).json()
results1

results2 = requests.get(url2).json()
results2

results3 = requests.get(url3).json()
results3

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

### first city ###    
    
venues1 = results1['response']['groups'][0]['items']
nearby_venues1 = json_normalize(venues1) # flatten JSON

# filter columns
filtered_columns1 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues1 =nearby_venues1.loc[:, filtered_columns1]

# filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)

# clean columns
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]


### second city ###

venues2 = results2['response']['groups'][0]['items']
nearby_venues2 = json_normalize(venues2) # flatten JSON

# filter columns
filtered_columns2 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues2 =nearby_venues2.loc[:, filtered_columns2]

# filter the category for each row
nearby_venues2['venue.categories'] = nearby_venues2.apply(get_category_type, axis=1)

# clean columns
nearby_venues2.columns = [col.split(".")[-1] for col in nearby_venues2.columns]


### third city ###

venues3 = results3['response']['groups'][0]['items']
nearby_venues3 = json_normalize(venues3) # flatten JSON

# filter columns
filtered_columns3 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues3 =nearby_venues3.loc[:, filtered_columns3]

# filter the category for each row
nearby_venues3['venue.categories'] = nearby_venues3.apply(get_category_type, axis=1)

# clean columns
nearby_venues3.columns = [col.split(".")[-1] for col in nearby_venues3.columns]





print('{} venues were returned by Foursquare.'.format(nearby_venues1.shape[0]))
print('{} venues were returned by Foursquare.'.format(nearby_venues2.shape[0]))
print('{} venues were returned by Foursquare.'.format(nearby_venues3.shape[0]))


100 venues were returned by Foursquare.
100 venues were returned by Foursquare.
100 venues were returned by Foursquare.


In [5]:
# add locations data to the data sets of each city

nearby_venues1['city'] = 'San Francisco'
nearby_venues2['city'] = 'Los Angeles'
nearby_venues3['city'] = 'San Diego'

In [6]:
# combine the three cities into one data set

nearby_venues = nearby_venues1.copy()
nearby_venues = nearby_venues.append(nearby_venues2)
nearby_venues = nearby_venues.append(nearby_venues3)


In [7]:
# check list and manually remove all non-restaurant data

nearby_venues['categories'].unique()

removal_list = ['Concert Hall', 'Opera House', 'Dance Studio',
                'Performing Arts Venue', 'Art Museum', 'Park',
                'Massage Studio', 'Music Venue', 'Bookstore', 'Clothing Store',
                'Boutique', 'Furniture/Home Store', 'Jazz Club',
                'Theater', 'Optical Shop', "Men's Store", 'Rock Club',
                'Gym / Fitness Center', 'Wine Shop', 'Indie Movie Theater',
                'Chocolate Shop', 'Dessert Shop', 'Recreation Center', 
                'Plaza', 'Hotel', 'Luggage Store', 'Farmers Market', 'Gym',
                'Jewelry Store', 'Furniture / Home Store', 'Butcher', 
                'Bakery', 'Marijuana Dispensary', 'Ice Cream Shop',
                'Comic Shop', 'Bagel Shop', 'Spa', 'Liquor Store', 'Bike Shop',
                'Yoga Studio', 'Pedestrian Plaza', 'Candy Store',
                'Park', 'Bookstore', 'Candy Store',  'Jazz Club', 'Art Gallery', 
                 'Supermarket', 'Museum', 'Boutique', 'Plaza', 'Building', 'Bakery',
                 'Historic Site', 'Ice Cream Shop', ' Concert Hall', 'Pharmacy', 
                 'Market', 'Movie Theater', 'Performing Arts Venue', 'Music Venue',
                 'Theater', 'Art Museum', 'Cheese Shop', 'Opera House',
                 'Pedestrian Plaza', 'School', 'Gift Shop', 'Athletics & Sports',
                 'Shoe Repair', 'General Entertainment', 'Stationery Store',
                 'Toy / Game Store', 'Brewery', 'Hotel', 'Theater', 'Music Venue', 'Business Service',
                 'Donut Shop', 'Liquor Store', 'Beer Store',
                 'Lounge', 'Plaza', 'Health Food Store', 'Concert Hall', 
                 'Lingerie Store', 'Gym', 'Mobile Phone Shop',
                 'Chocolate Shop', 'Ice Cream Shop', 'Hostel', 'Convenience Store', 
                 'Park', 'Farmers Market', 'Cosmetics Shop', 'Piano Bar',
                 'Nightclub', 'Massage Studio', 'Comedy Club', 'Concert Hall']

nearby_venues = nearby_venues[~nearby_venues['categories'].isin(removal_list)]

nearby_venues['categories'].unique()               

array(['French Restaurant', 'Coffee Shop', 'Scandinavian Restaurant',
       'Beer Bar', 'Cocktail Bar', 'Ramen Restaurant',
       'Vegetarian / Vegan Restaurant', 'Tiki Bar', 'Café',
       'Sushi Restaurant', 'Mexican Restaurant',
       'Southern / Soul Food Restaurant', 'Vietnamese Restaurant',
       'Wine Bar', 'Thai Restaurant', 'Food & Drink Shop',
       'Sandwich Place', 'Food Truck', 'Poke Place', 'Juice Bar',
       'Italian Restaurant', 'Hotel Bar', 'Restaurant', 'Taco Place',
       'Beer Garden', 'New American Restaurant', 'Souvlaki Shop',
       'Frozen Yogurt Shop', 'Indian Restaurant', 'American Restaurant',
       'Japanese Curry Restaurant', 'Indonesian Restaurant',
       'Breakfast Spot', 'Speakeasy', 'Udon Restaurant',
       'Japanese Restaurant', 'Bar', 'Noodle House', 'Deli / Bodega',
       'BBQ Joint', 'Falafel Restaurant', 'Yoshoku Restaurant',
       'Filipino Restaurant', 'Latin American Restaurant',
       'Seafood Restaurant', 'Mediterranean Restaurant

In [8]:
# set up to pull the likes from the API based on venue ID

url_list = []
like_list = []
json_list = []

for i in list(nearby_venues.id):
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
    url_list.append(venue_url)
for link in url_list:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    like_list.append(likes)
print(like_list)


nearby_venues['likes'] = like_list
nearby_venues.head()

[234, 475, 194, 286, 583, 328, 145, 452, 1099, 690, 133, 288, 1162, 167, 357, 281, 357, 94, 667, 518, 214, 86, 426, 931, 115, 112, 35, 69, 454, 28, 306, 62, 78, 801, 460, 53, 59, 79, 266, 47, 166, 790, 175, 1034, 579, 310, 136, 203, 274, 291, 157, 716, 207, 207, 125, 17, 122, 68, 305, 340, 340, 573, 70, 320, 121, 105, 162, 719, 14, 89, 78, 105, 42, 63, 377, 12, 73, 78, 103, 9, 30, 117, 391, 111, 155, 193, 10, 50, 35, 349, 25, 111, 17, 37, 25, 56, 112, 41, 32, 318, 10, 472, 154, 40, 108, 36, 248, 55, 150, 70, 8, 58, 147, 166, 33, 93, 128, 34, 202, 142, 53, 26, 35, 9, 103, 62, 24, 11, 13, 57, 31, 18, 473, 119, 8, 28, 301, 8, 218, 7, 28, 74, 55, 48, 284, 28, 98, 61, 26, 25, 95, 34, 34, 19, 96, 435, 307, 513, 456, 97, 179, 65, 7, 10, 189, 463, 430, 127]


,name,categories,lat,lng,id,city,likes
5,Jardinière,French Restaurant,37.778042,-122.421725,49d3e017f964a5201e5c1fe3,San Francisco,234
8,Philz Coffee,Coffee Shop,37.781433,-122.417073,5151a10ce4b06ae7735335db,San Francisco,475
10,PLÄJ,Scandinavian Restaurant,37.778733,-122.422106,4fe942b1e4b0f69d225327ad,San Francisco,194
11,The Beer Hall,Beer Bar,37.776837,-122.417916,519fcb37498e91a13cb23d6b,San Francisco,286
12,Whitechapel,Cocktail Bar,37.782230,-122.418884,55df9e27498ef4c72ac83a77,San Francisco,583


In [9]:
# this is really the raw dataset now so let us rename it something more appropriate

raw_dataset = nearby_venues
raw_dataset.head()

,name,categories,lat,lng,id,city,likes
5,Jardinière,French Restaurant,37.778042,-122.421725,49d3e017f964a5201e5c1fe3,San Francisco,234
8,Philz Coffee,Coffee Shop,37.781433,-122.417073,5151a10ce4b06ae7735335db,San Francisco,475
10,PLÄJ,Scandinavian Restaurant,37.778733,-122.422106,4fe942b1e4b0f69d225327ad,San Francisco,194
11,The Beer Hall,Beer Bar,37.776837,-122.417916,519fcb37498e91a13cb23d6b,San Francisco,286
12,Whitechapel,Cocktail Bar,37.782230,-122.418884,55df9e27498ef4c72ac83a77,San Francisco,583


## 2.2 Data Preparation

The data still needs some more processing before it is suitable for model training and testing. Mainly, the "categories" column contains too many different types of cuisines to allow a model to yield any meaningful results. However, the different types of natural cuisines have natural groupings based on conventionally accepted cultural groupings of cuisine. Broadly speaking, all of the different types of cuisine could be reclassified as European, Latin American, Asian, North American, drinking establishments (bars), or casual establishments such as coffee shops or ice cream parlours. We can implement manual classification as there really aren't that many different types of cuisines.

As this project will compare both linear and logistic regression, it makes sense to have "likes" as both a continuous and categorical (but ordinal) variable. In the case of turning into a categorical variable, we can bin the data based on percentiles and classify them into these ordinal percentile categories. I tried different ways of binning but in the end, splitting the sample into three different bins proved to yield the best classification results from a prediction standpoint.

As the last stage of data preparation, it is important to note that the regressors are categorical variables (3 different cities and 6 different categories of cusines). Hence, they require dummy variable encoding for meaningful analysis. We can accomplish this via one-hot encoding. 

In [10]:
# inspecting the raw dataset shows that there may be too many different types of cuisines
raw_dataset['categories'].unique()

array(['French Restaurant', 'Coffee Shop', 'Scandinavian Restaurant',
       'Beer Bar', 'Cocktail Bar', 'Ramen Restaurant',
       'Vegetarian / Vegan Restaurant', 'Tiki Bar', 'Café',
       'Sushi Restaurant', 'Mexican Restaurant',
       'Southern / Soul Food Restaurant', 'Vietnamese Restaurant',
       'Wine Bar', 'Thai Restaurant', 'Food & Drink Shop',
       'Sandwich Place', 'Food Truck', 'Poke Place', 'Juice Bar',
       'Italian Restaurant', 'Hotel Bar', 'Restaurant', 'Taco Place',
       'Beer Garden', 'New American Restaurant', 'Souvlaki Shop',
       'Frozen Yogurt Shop', 'Indian Restaurant', 'American Restaurant',
       'Japanese Curry Restaurant', 'Indonesian Restaurant',
       'Breakfast Spot', 'Speakeasy', 'Udon Restaurant',
       'Japanese Restaurant', 'Bar', 'Noodle House', 'Deli / Bodega',
       'BBQ Joint', 'Falafel Restaurant', 'Yoshoku Restaurant',
       'Filipino Restaurant', 'Latin American Restaurant',
       'Seafood Restaurant', 'Mediterranean Restaurant

In [11]:
# we can group some cuisines together to make a better categorical variable

euro = ['French Restaurant', 'Scandinavian Restaurant', 'Souvlaki Shop', 
       'Mediterranean Restaurant', 'Italian Restaurant', 'Pizza Place']

latino = ['Mexican Restaurant', 'Latin American Restaurant', 
          'Brazilian Restaurant', 'Taco Place']

bar = ['Beer Bar', 'Cocktail Bar', 'Tiki Bar', 'Wine Bar', 'Hotel Bar',
       'Beer Garden', 'Speakeasy', 'Brewery', 'Pub', 'Bar', 'Gastropub',
       'Hookah Bar']

asian = ['Ramen Restaurant', 'Sushi Restaurant', 'Vietnamese Restaurant',
         'Thai Restaurant', 'Poke Place', 'Indian Restaurant', 
         'Japanese Curry Restaurant', 'Japanese Restaurant', 
         'Indonesian Restaurant', 'Udon Restaurant', 'Noodle House',
         'Falafel Restaurant', 'Filipino Restaurant', 'Turkish Restaurant',
         'Yoshoku Restaurant']

casual = ['Coffee Shop', 'Café', 'Sandwich Place', 'Food Truck',
          'Juice Bar', 'Frozen Yogurt Shop', 'Deli / Bodega', 'Dessert Shop',
          'Hot Dog Joint', 'Burger Joint', 'Breakfast Spot', 
          'Fondue Restaurant']

american = ['Southern / Soul Food Restaurant', 'Food & Drink Shop', 
            'Restaurant', 'American Restaurant', 'BBQ Joint', 
            'Theme Restaurant', 'New American Restaurant',
            'Vegetarian / Vegan Restaurant', 'Seafood Restaurant']

def conditions(s):
    if s['categories'] in euro:
        return 'euro'
    if s['categories'] in latino:
        return 'latino'
    if s['categories'] in asian:
        return 'asian'
    if s['categories'] in casual:
        return 'casual'
    if s['categories'] in american:
        return 'american'
    if s['categories'] in bar:
        return 'bar'


raw_dataset['categories_classified']=raw_dataset.apply(conditions, axis=1)
raw_dataset

,name,categories,lat,lng,id,city,likes,categories_classified
5,Jardinière,French Restaurant,37.778042,-122.421725,49d3e017f964a5201e5c1fe3,San Francisco,234,euro
8,Philz Coffee,Coffee Shop,37.781433,-122.417073,5151a10ce4b06ae7735335db,San Francisco,475,casual
10,PLÄJ,Scandinavian Restaurant,37.778733,-122.422106,4fe942b1e4b0f69d225327ad,San Francisco,194,euro
11,The Beer Hall,Beer Bar,37.776837,-122.417916,519fcb37498e91a13cb23d6b,San Francisco,286,bar
12,Whitechapel,Cocktail Bar,37.782230,-122.418884,55df9e27498ef4c72ac83a77,San Francisco,583,bar
14,Nojo Ramen Tavern,Ramen Restaurant,37.776637,-122.421270,4d8eabc7d265236af9a71017,San Francisco,328,asian
15,Ananda Fuara,Vegetarian / Vegan Restaurant,37.777693,-122.416353,4a0cb0fff964a52056751fe3,San Francisco,145,american
19,Blue Bottle Coffee,Coffee Shop,37.776286,-122.416867,5560dbdb498e91a2bcde84f6,San Francisco,452,casual
23,Smuggler's Cove,Tiki Bar,37.779386,-122.423422,4afe6db4f964a520682f22e3,San Francisco,1099,bar
24,Arlequin Cafe & Food To Go,Café,37.777014,-122.422639,49eb8289f964a520ea661fe3,San Francisco,690,casual


In [12]:
# double check to make sure categories_classified has been created correctly

pd.crosstab(index=raw_dataset["categories_classified"],
            columns="count")  

col_0,count
categories_classified,
american,24
asian,40
bar,30
casual,39
euro,18
latino,17


In [ ]:
# classify the likes into different ranking levels
# lets first see where to bin the data
# we can try different ways of binning the data, I find it yields substantially different results

# print(np.percentile(raw_dataset['likes'], 20))
# print(np.percentile(raw_dataset['likes'], 40))
# print(np.percentile(raw_dataset['likes'], 60))
# print(np.percentile(raw_dataset['likes'], 80))


print(np.percentile(raw_dataset['likes'], 33))
print(np.percentile(raw_dataset['likes'], 66))


# print(np.percentile(raw_dataset['likes'], 50))

In [23]:
# create a function to bin for us

# def rankings(s):
#     if s['likes']<=34:
#         return 1
#     if s['likes']<=78:
#         return 2
#     if s['likes']<=154.6:
#         return 3
#     if s['likes']<=326.4:
#         return 4
#     if s['likes']>326.4:
#         return 5

def rankings(s):
    if s['likes']<=62:
        return 1
    if s['likes']<=193.22:
        return 2
    if s['likes']>193.22:
        return 3
    
    
# def rankings(s):
#     if s['likes']<=111.5:
#         return 0
#     if s['likes']>111.5:
#         return 1

raw_dataset['ranking']=raw_dataset.apply(rankings, axis=1)
raw_dataset

,name,categories,lat,lng,id,city,likes,categories_classified,ranking
5,Jardinière,French Restaurant,37.778042,-122.421725,49d3e017f964a5201e5c1fe3,San Francisco,234,euro,3
8,Philz Coffee,Coffee Shop,37.781433,-122.417073,5151a10ce4b06ae7735335db,San Francisco,475,casual,3
10,PLÄJ,Scandinavian Restaurant,37.778733,-122.422106,4fe942b1e4b0f69d225327ad,San Francisco,194,euro,3
11,The Beer Hall,Beer Bar,37.776837,-122.417916,519fcb37498e91a13cb23d6b,San Francisco,286,bar,3
12,Whitechapel,Cocktail Bar,37.782230,-122.418884,55df9e27498ef4c72ac83a77,San Francisco,583,bar,3
14,Nojo Ramen Tavern,Ramen Restaurant,37.776637,-122.421270,4d8eabc7d265236af9a71017,San Francisco,328,asian,3
15,Ananda Fuara,Vegetarian / Vegan Restaurant,37.777693,-122.416353,4a0cb0fff964a52056751fe3,San Francisco,145,american,2
19,Blue Bottle Coffee,Coffee Shop,37.776286,-122.416867,5560dbdb498e91a2bcde84f6,San Francisco,452,casual,3
23,Smuggler's Cove,Tiki Bar,37.779386,-122.423422,4afe6db4f964a520682f22e3,San Francisco,1099,bar,3
24,Arlequin Cafe & Food To Go,Café,37.777014,-122.422639,49eb8289f964a520ea661fe3,San Francisco,690,casual,3


In [24]:
# create dummies for linear regression modelling

# one hot encoding
reg_dataset = pd.get_dummies(raw_dataset[['categories_classified', 
                                          'city',]], 
                               prefix="", 
                               prefix_sep="")

# add name, ranking, and likes columns back to dataframe
reg_dataset['ranking'] = raw_dataset['ranking']
reg_dataset['likes'] = raw_dataset['likes']
reg_dataset['name'] = raw_dataset['name']

# move name column to the first column
reg_columns = [reg_dataset.columns[-1]] + list(reg_dataset.columns[:-1])
reg_dataset = reg_dataset[reg_columns]


reg_dataset.head()

,name,american,asian,bar,casual,euro,latino,Los Angeles,San Diego,San Francisco,ranking,likes
5,Jardinière,0,0,0,0,1,0,0,0,1,3,234
8,Philz Coffee,0,0,0,1,0,0,0,0,1,3,475
10,PLÄJ,0,0,0,0,1,0,0,0,1,3,194
11,The Beer Hall,0,0,1,0,0,0,0,0,1,3,286
12,Whitechapel,0,0,1,0,0,0,0,0,1,3,583


# 3. Methodology

This project will utilize both linear and logistic regression machine learning methods to train and test the data. Namely, linear regression will be used in an attempt to predict the number of "likes" a new restaurant in this region will have. We will utilize the Sci-Kit Learn Package to run the model. 

We can also utilize logisitc regression as a classification method rather than direct prediction of the number of likes. Since the number of "likes" can be binned into different categories based on different percentile bins, it is also potentiallly possible to see which range of "likes" a new restaurant in this region will have. 

Since the "likes" are binned into multiple (more than 2) categories, the type of logistic regression will be multinomial. Additionally, although the ranges are indeed discrete categories, they are also ordinal in nature. Therefore the logistic regression will need to be specified as being both multinomial and ordinal. This can be done through the Sci-Kit Learn Package as well.

# 4. Results 

## 4.1 Linear Regression Results

A linear regression model was trained on a random subsample of 80% of the sample and then tested on the other 20%. To see if this is a reasonable model. the residual sum of squares score and variance score were both calculated (36805.64 and 0.01 respectively). Given the low variance score, this is probably not a valid/good way of modelling the data. Therefore, we move on to logistic regression.

In [63]:
# Multiple Linear Regression

msk = np.random.rand(len(reg_dataset)) < 0.8
train = reg_dataset[msk]
test = reg_dataset[~msk]

regr = linear_model.LinearRegression()
x = np.asanyarray(train[['american', 'asian', 'bar', 'casual',
                         'euro', 'latino', 'Los Angeles', 
                         'San Diego', 'San Francisco']])
y = np.asanyarray(train[['likes']])
regr.fit (x, y)
# The coefficients
print ('Coefficients: ', regr.coef_)

Coefficients:  [[ 63.38198655 -37.56110552  52.85484068 -56.43098657  35.66981518
  -57.9145503  -54.78882794 -82.242033   137.03086094]]


In [59]:
# Multiple Linear Regression Prediction Capabilities

y_hat= regr.predict(test[['american', 'asian', 'bar', 'casual',
                         'euro', 'latino', 'Los Angeles', 
                         'San Diego', 'San Francisco']])
x = np.asanyarray(test[['american', 'asian', 'bar', 'casual',
                         'euro', 'latino', 'Los Angeles', 
                         'San Diego', 'San Francisco']])
y = np.asanyarray(test[['likes']])
print("Residual sum of squares: %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))

Residual sum of squares: 36805.64
Variance score: 0.01


## 4.2 Logistic Regression Results

A multinomial ordinal logisitc regression model was trained on a random subsample of 80% of the sample and then tested on the other 20%. To see if this is a reasonable model, its jaccard similarity score and log-loss were calculated (66.66% and 1.009 respectively). Although this is not a perfect prediction, a similarity of 66% between the training set and test set is a reasonable result. The classification report is also printed later on below.
<br>

Given the modestly accurate ability of this model, we can also run the model on the full dataset. The coefficients show that opening a restaurant in San Francisco, opening a bar, or serving cuisine that is american or asian in nature, are associated negatively with "likes." 

In [47]:
# Multinomial Ordinal Logistic Regression

x_train = np.asanyarray(train[['american', 'asian', 'bar', 'casual',
                         'euro', 'latino', 'Los Angeles', 
                         'San Diego', 'San Francisco']])
y_train = np.asanyarray(train['ranking'])

x_test = np.asanyarray(test[['american', 'asian', 'bar', 'casual',
                         'euro', 'latino', 'Los Angeles', 
                         'San Diego', 'San Francisco']])
y_test = np.asanyarray(test['ranking'])


# LR = LogisticRegression(C=0.01, solver='liblinear').fit(x_train, y_train)
# LR

mul_ordinal = linear_model.LogisticRegression(multi_class='multinomial',
                                              solver='newton-cg',
                                              fit_intercept=True).fit(x_train,
                                                                      y_train)

mul_ordinal

coef = mul_ordinal.coef_[0]
print (coef)

[-0.16660509  0.0894391  -0.61702958  0.24421489  0.04084001  0.40914038
  0.23013575  0.45851481 -0.68865086]


In [48]:
# Multinomial Ordinal Logistic Regression Prediction Capabilities

yhat = mul_ordinal.predict(x_test)
yhat

yhat_prob = mul_ordinal.predict_proba(x_test)
yhat_prob


jaccard_similarity_score(y_test, yhat)

0.6666666666666666

In [49]:
log_loss(y_test, yhat_prob)

1.0095339584386367

In [50]:
# Exploration of Coefficient Magnitudes of Full Dataset

x_all = np.asanyarray(reg_dataset[['american', 'asian', 'bar', 'casual',
                                   'euro', 'latino', 'Los Angeles', 
                                   'San Diego', 'San Francisco']])
y_all = np.asanyarray(reg_dataset['ranking'])



LR = linear_model.LogisticRegression(multi_class='multinomial',
                                            solver='newton-cg',
                                            fit_intercept=True).fit(x_all,
                                                                    y_all)

LR

coef = LR.coef_[0]
print (coef)

[-0.18692265 -0.07515606 -0.54222379  0.44062699  0.02190218  0.34177331
  0.12475147  0.55969312 -0.6844446 ]


In [51]:
print (classification_report(y_test, yhat))

              precision    recall  f1-score   support

           1       0.80      0.80      0.80        10
           2       0.54      0.78      0.64         9
           3       0.71      0.45      0.56        11

   micro avg       0.67      0.67      0.67        30
   macro avg       0.68      0.68      0.66        30
weighted avg       0.69      0.67      0.66        30



## 5. Discussion

The first thing to note is that given the data, logistic regression presents a better fit for the data over linear regression. Using logistic regression we were able to obtain a Jaccard Similarity Score of 66.66%, which although not perfect, is more reasonable than the low variance score obtained from the linear regression. As stated before, please note that for the purposes of this project, we are assumming that likes are a good proxy for how well a new restaurant will do in terms of brand, image and by extension how well the restaurant will perform business-wise. Whether or not these assumptions hold up in a real-life scenario is up for discussion, but this project does contain limitations in scope due to the amount of data that can be fetched from the FourSquare API. 

<br>

As such, to obtain insights into this data, we can proceed with breaking down the results of the logistic regression model. The results showed that the precision score for classifying whether the new restaurant would fall into classes 1, 2, or 3 (lowest, medium, or highest percentile of likes) were 80%, 54%, and 71%. Therefore, the model is better at predicting if a restaurant will fall into the best or worst percentile of likes. This is good as we are mostly concerned with whether the restuarant will perform well or not so the high accuracy of predictions for the two extremum is a welcome feature. This allows us to fairly accurately predict the general performance of the business opportunity. Different binning methods for the classes were attempted, but the use of 3 bins by far yielded the best Jaccard Similarity Score. 

<br>

Additionally, not only are we attempting to predict the general business performance but also pull insights to inform on business strategy. In this case strategy insight can be gleamed from the coefficient values from running the logistic regressin on the full dataset. As such, we can see that opening a restaurant in San Francisco, opening a bar, or serving cuisine that is american or asian in nature, are associated negatively with "likes." This suggests that the business opportunity should be opening a restaurant in either Los Angeles or San Diego, with a cuisine that is European, Latino, or casual in nature would be the best approach for maximizing likes. 

## 6. Conclusion

In conclusion, after analyzing restaurant "likes" in California from 300 restaurants, we can conclude that the approach to best take in regards to maximizing business performance (as measured by "likes") is to open a restaurant that is either European, Latino, or casual and that opening the venue in either Los Angeles or San Diego rather than San Francisco would be the best approach. Additionally, the predictive capabilities of the logistic regression prediction model are most accurate for classifying whether a restaurant will fall in either the best or worst classes when the data is binned into 3 classes. 